In [6]:
DATASET = 'iris'
ARTIFACT_PATH = '/User/functions/get_toy_data/artifacts/{{run.uid}}'

pkg_module = 'sklearn.datasets'
fname = f'load_{DATASET}'

pkg_module = __import__(pkg_module, fromlist=[fname])
load_data_fn = getattr(pkg_module, fname)

data = load_data_fn()

data['feature_names']

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [7]:
import mlrun
import os

mlrun.mlconf.dbpath = 'http://mlrun-api:8080'
mlrun.mlconf.hub_url = 'https://raw.githubusercontent.com/yjb-ds/functions/init_fixes/{name}/function.yaml'

In [8]:
FUNCTION = 'get_toy_data'
urls = [
    'function.yaml',
    f'db://{FUNCTION}:latest',
    f'hub://{FUNCTION}'
]

In [9]:
for url in urls:
    try:
        rfn = mlrun.import_function(url).apply(mlrun.mount_v3io())
        tsk = rfn.run(
                mlrun.NewTask(
                    "load_data", 
                    params={"dataset"     : DATASET},
                    artifact_path=ARTIFACT_PATH))
    except Exception as e:
        print(url, str(e))
    print("++++++++++++++++++++++++++++++++++++++++++++++++")

[mlrun] 2020-03-16 00:04:17,286 warning!, server (0.4.5) and client (0.4.6) ver dont match
[mlrun] 2020-03-16 00:04:17,287 starting run load_data uid=43ac52f6f899428b9894718bf581bb69  -> http://mlrun-api:8080
[mlrun] 2020-03-16 00:04:17,366 Job is running in the background, pod: load-data-8cgth
[mlrun] 2020-03-16 00:04:22,419 log artifact iris at /User/functions/get_toy_data/artifacts/43ac52f6f899428b9894718bf581bb69/iris.pqt, size: None, db: Y

[mlrun] 2020-03-16 00:04:22,430 run executed, status=completed
Intel(R) Data Analytics Acceleration Library (Intel(R) DAAL) solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html
final state: succeeded


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...81bb69,0,Mar 16 00:04:21,completed,load_data,host=load-data-8cgthkind=jobowner=admin,,dataset=iris,,iris


to track results use .show() or .logs() or in CLI: 
!mlrun get run 43ac52f6f899428b9894718bf581bb69  , !mlrun logs 43ac52f6f899428b9894718bf581bb69 
[mlrun] 2020-03-16 00:04:23,516 run executed, status=completed
++++++++++++++++++++++++++++++++++++++++++++++++


/User/.pythonlibs/jupyter/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


[mlrun] 2020-03-16 00:04:23,813 warning!, server (0.4.5) and client (0.4.6) ver dont match
[mlrun] 2020-03-16 00:04:23,814 starting run load_data uid=f95d4ca2b2e14d82b0344d477505829e  -> http://mlrun-api:8080
[mlrun] 2020-03-16 00:04:24,059 Job is running in the background, pod: load-data-d9p6d
[mlrun] 2020-03-16 00:04:28,983 log artifact iris at /User/functions/get_toy_data/artifacts/f95d4ca2b2e14d82b0344d477505829e/iris.pqt, size: None, db: Y

[mlrun] 2020-03-16 00:04:28,993 run executed, status=completed
Intel(R) Data Analytics Acceleration Library (Intel(R) DAAL) solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html
final state: succeeded


...

to track results use .show() or .logs() or in CLI: 
!mlrun get run f95d4ca2b2e14d82b0344d477505829e  , !mlrun logs f95d4ca2b2e14d82b0344d477505829e 
[mlrun] 2020-03-16 00:04:30,186 run executed, status=completed
++++++++++++++++++++++++++++++++++++++++++++++++
[mlrun] 2020-03-16 00:04:30,193 warning!, server (0.4.5) and client (0.4.6) ver dont match
db://get_toy_data:latest 'function get_toy_data:latest not found in the DB'
++++++++++++++++++++++++++++++++++++++++++++++++
[mlrun] 2020-03-16 00:04:30,271 warning!, server (0.4.5) and client (0.4.6) ver dont match
[mlrun] 2020-03-16 00:04:30,272 starting run load_data uid=cb76b913e56d4f508767eee2558ffffa  -> http://mlrun-api:8080
[mlrun] 2020-03-16 00:04:30,347 Job is running in the background, pod: load-data-frj7l


/User/.pythonlibs/jupyter/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


[mlrun] 2020-03-16 00:04:35,372 log artifact iris at /User/functions/get_toy_data/artifacts/cb76b913e56d4f508767eee2558ffffa/iris.pqt, size: None, db: Y

[mlrun] 2020-03-16 00:04:35,386 run executed, status=completed
Intel(R) Data Analytics Acceleration Library (Intel(R) DAAL) solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html
final state: succeeded


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...8ffffa,0,Mar 16 00:04:34,completed,load_data,host=load-data-frj7lkind=jobowner=admin,,dataset=iris,,iris


to track results use .show() or .logs() or in CLI: 
!mlrun get run cb76b913e56d4f508767eee2558ffffa  , !mlrun logs cb76b913e56d4f508767eee2558ffffa 
[mlrun] 2020-03-16 00:04:36,448 run executed, status=completed
++++++++++++++++++++++++++++++++++++++++++++++++
